In [1]:
!pip install --upgrade pymatgen biopython mp-api

In [19]:
from mp_api.client import MPRester
from pymatgen.core import Composition, Structure
import re
import os
from Bio.Seq import Seq
import requests
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.ExPASy import get_sprot_raw
from Bio.SwissProt import read
from collections import Counter
import itertools

# Часть 1: Материалы

In [7]:
# Загружаем API ключ
api_key = os.environ.get("PMG_MAPI_KEY")
if not api_key:
    api_key = input("Введите ваш API ключ от Materials Project: ")
    os.environ["PMG_MAPI_KEY"] = api_key

with MPRester(api_key) as m:
    # Поиск материала по формуле
    docs = m.materials.summary.search(formula="MnFe2O4", fields=["material_id", "formula_pretty"])

    if not docs:
        raise ValueError("Материал MnFe2O4 не найден в базе Materials Project.")

    material_id = docs[0].material_id
    print(f"Найден материал: {docs[0].formula_pretty}, ID: {material_id}")

    # Загрузка структуры
    structure = m.get_structure_by_material_id(material_id)
    print(" Структура успешно загружена!")

    # Анализ состава
    composition = structure.composition
    print("\nСостав:", composition)
    print("Элементы:", [el.symbol for el in composition.elements])

    # Параметры решётки
    lattice = structure.lattice
    print("\nПараметры решётки:")
    print(f"a = {lattice.a:.4f} Å, b = {lattice.b:.4f} Å, c = {lattice.c:.4f} Å")
    print(f"α = {lattice.alpha:.2f}°, β = {lattice.beta:.2f}°, γ = {lattice.gamma:.2f}°")

    # Плотность
    density = structure.density
    print(f"\nПлотность: {density:.4f} г/см³")


Retrieving SummaryDoc documents:   0%|          | 0/6 [00:00<?, ?it/s]

Найден материал: Mn(FeO2)2, ID: mp-697842


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

 Структура успешно загружена!

Состав: Mn10 Fe20 O40
Элементы: ['Mn', 'Fe', 'O']

Параметры решётки:
a = 6.1505 Å, b = 6.1007 Å, c = 30.3960 Å
α = 60.28°, β = 60.00°, γ = 59.84°

Плотность: 4.7446 г/см³


# Часть 2: Последовательности ДНК/РНК

In [22]:
# ДНК-последовательности
sequences = {
    "BRCA1": Seq("ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTA"),
    "TP53": Seq("ATGGAAGCTCCCAGAATGCCAGAGGCTGCTCCCCCAGGGAGCACCT")
}

for name, seq in sequences.items():
    print(f"\n{name}: {seq}")

    # GC-состав
    gc_content = 100 * (seq.count("G") + seq.count("C")) / len(seq)
    print(f"GC-состав: {gc_content:.2f}%")

    # Обратная комплементарная
    rev_comp = seq.reverse_complement()
    print(f"Обратная комплементарная: {rev_comp}")

    # Транскрипция в РНК
    rna = seq.transcribe()
    print(f"РНК: {rna}")


BRCA1: ATGGATTTATCTGCTCTTCGCGTTGAAGAAGTACAAAATGTCATTA
GC-состав: 34.78%
Обратная комплементарная: TAATGACATTTTGTACTTCTTCAACGCGAAGAGCAGATAAATCCAT
РНК: AUGGAUUUAUCUGCUCUUCGCGUUGAAGAAGUACAAAAUGUCAUUA

TP53: ATGGAAGCTCCCAGAATGCCAGAGGCTGCTCCCCCAGGGAGCACCT
GC-состав: 63.04%
Обратная комплементарная: AGGTGCTCCCTGGGGGAGCAGCCTCTGGCATTCTGGGAGCTTCCAT
РНК: AUGGAAGCUCCCAGAAUGCCAGAGGCUGCUCCCCCAGGGAGCACCU


# Часть 3: Белки

In [16]:
# Получение данных по альфа-Амилазе (UniProt ID: P00691)
handle = get_sprot_raw("P00691")
record = read(handle)
aa_sequence = record.sequence

print(f"Последовательность амилазы:\n{aa_sequence}\n")

# Состав аминокислот
aa_counts = Counter(aa_sequence)
print("Состав аминокислот:")
for aa, count in sorted(aa_counts.items()):
    print(f"{aa}: {count}")

# Частоты дипептидов
dipeptides = [aa_sequence[i:i+2] for i in range(len(aa_sequence)-1)]
dipeptide_counts = Counter(dipeptides)

print("\nТоп-10 дипептидов:")
for dipep, count in dipeptide_counts.most_common(10):
    print(f"{dipep}: {count}")

Последовательность амилазы:
MFAKRFKTSLLPLFAGFLLLFHLVLAGPAAASAETANKSNELTAPSIKSGTILHAWNWSFNTLKHNMKDIHDAGYTAIQTSPINQVKEGNQGDKSMSNWYWLYQPTSYQIGNRYLGTEQEFKEMCAAAEEYGIKVIVDAVINHTTSDYAAISNEVKSIPNWTHGNTQIKNWSDRWDVTQNSLLGLYDWNTQNTQVQSYLKRFLDRALNDGADGFRFDAAKHIELPDDGSYGSQFWPNITNTSAEFQYGEILQDSASRDAAYANYMDVTASNYGHSIRSALKNRNLGVSNISHYASDVSADKLVTWVESHDTYANDDEESTWMSDDDIRLGWAVIASRSGSTPLFFSRPEGGGNGVRFPGKSQIGDRGSALFEDQAITAVNRFHNVMAGQPEELSNPNGNNQIFMNQRGSHGVVLANAGSSSVSINTATKLPDGRYDNKAGAGSFQVNDGKLTGTINARSVAVLYPDDIAKAPHVFLENYKTGVTHSFNDQLTITLRADANTTKAVYQINNGPETAFKDGDQFTIGKGDPFGKTYTIMLKGTNSDGVTRTEKYSFVKRDPASAKTIGYQNPNHWSQVNAYIYKHDGSRVIELTGSWPGKPMTKNADGIYTLTLPADTDTTNAKVIFNNGSAQVPGQNQPGFDYVLNGLYNDSGLSGSLPH

Состав аминокислот:
A: 58
C: 1
D: 45
E: 23
F: 27
G: 56
H: 17
I: 34
K: 34
L: 43
M: 10
N: 55
P: 26
Q: 28
R: 22
S: 56
T: 49
V: 33
W: 14
Y: 28

Топ-10 дипептидов:
GS: 11
DG: 9
SA: 8
AG: 7
AA: 7
TA: 7
NT: 7
AS: 6
SN: 6
TI: 6
